In [6]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import Pipeline
from scipy import sparse
from sklearn.svm import LinearSVC
import json
from sklearn.model_selection import train_test_split
import copy

In [10]:
with open('dictionaries/train.json', encoding="UTF-8") as f:
    raw_train = json.load(f)

In [11]:
len(raw_train)

8263

In [27]:
ru_token(raw_train[0]["text"])

['Досудебное',
 'расследование',
 'по',
 'факту',
 'покупки',
 'ЕНПФ',
 'пакета',
 'облигаций',
 'ТОО',
 'Бузгул',
 'Аурум',
 'было',
 'начато',
 'по',
 'инициативе',
 'Национального',
 'банка',
 'РК',
 'сообщил',
 'директор',
 'департамента',
 'защиты',
 'прав',
 'потребителей',
 'и',
 'финансовых',
 'услуг',
 'Нацбанка',
 'Казахстана',
 'Александр',
 'Терентьев',
 'Основанием',
 'для',
 'досудебного',
 'расследования',
 'стало',
 'обращение',
 'Национального',
 'банка',
 'письмо',
 'от',
 'ноября',
 'года',
 'Было',
 'обращение',
 'Национального',
 'банка',
 'в',
 'правоохранительные',
 'органы',
 'Нам',
 'эта',
 'сделка',
 'показалась',
 'сомнительной',
 'недостаточно',
 'корректной',
 'поэтому',
 'Нацбанк',
 'ноября',
 'года',
 'обратился',
 'в',
 'правоохранительные',
 'органы',
 'Это',
 'то',
 'что',
 'я',
 'могу',
 'озвучить',
 'на',
 'сегодня',
 'Идёт',
 'следствие',
 'проводится',
 'проверка',
 'сказал',
 'Терентьев',
 'декабря',
 'в',
 'Нацбанке',
 'заявили',
 'что',
 'не',
 

In [12]:
def ru_token(string):
    """russian tokenize based on nltk.word_tokenize. only russian letter remaind."""
    return [i for i in word_tokenize(string) if re.match(r'[\u0400-\u04ffа́]+$', i)]

In [13]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), tokenizer=ru_token, stop_words=stopwords.words('russian'),
                        min_df=3,
                        use_idf=1, smooth_idf=1, sublinear_tf=1)

In [15]:
tfidf.fit([i['text'] for i in raw_train])

f:\programming\python\economanalyzer\venv\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(min_df=3, ngram_range=(1, 3), smooth_idf=1,
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                sublinear_tf=1,
                tokenizer=<function ru_token at 0x0000018707181A68>, use_idf=1)

In [57]:
class NBFeaturer(BaseEstimator, ClassifierMixin):
    """from https://www.kaggle.com/sermakarevich"""
    def __init__(self, alpha):
        self.alpha = alpha

    def preprocess_x(self, x, r):
        return x.multiply(r)

    def transform(self, x):
        x_nb = self.preprocess_x(x, self._r)
        return x_nb

    def fit(self, x, y):
        self._r = sparse.csr_matrix(np.log(self.pr(x, 1, y) / self.pr(x, 0, y)))
        return self

    def pr(self, x, y_i, y):
        p = x[y == y_i].sum(0)
        return (p + self.alpha) / ((y == y_i).sum() + self.alpha)

In [58]:
nbf = NBFeaturer(alpha=10)
model = LinearSVC(C=4, max_iter=30) # One-vs.-rest balanced category with 1:1:1

p = pipeline = Pipeline([
    ('nbf', nbf),
    ('lr', model)
])

In [59]:
train, val = train_test_split(raw_train, test_size=0.2, random_state=2018)

In [60]:
train_x = tfidf.transform([i['text'] for i in train])
lab = LabelEncoder()
lab.fit([i['sentiment'] for i in raw_train])
train_y = []
for i in range(3):
    train_y.append((lab.transform([d['sentiment'] for d in train]) == i).astype(int))
val_x = tfidf.transform([d['text'] for d in val])
val_y = lab.transform([d['sentiment'] for d in val])

In [61]:
a = lab.fit_transform([i['sentiment'] for i in raw_train])
w = [(len(a) - np.sum(a == i))/(2 * np.sum(a == i)) for i in range(max(a)+1)]

In [62]:
pred = []
test_pred = []
for i in range(3):
    p.get_params()['lr'].class_weight = {0: 1, 1:w[i] }
    p.fit(train_x, train_y[i])
    pred.append(p.decision_function(val_x))

f:\programming\python\economanalyzer\venv\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [63]:
accuracy_score(val_y, np.argmax(np.array(pred), axis=0))

0.6860254083484574

In [33]:
df = pd.read_csv("input2.csv", error_bad_lines=False, names=["link", "text", "date"])

In [70]:
df[["text", "result"]].head(50)

,text,result
0,В МЭР рассказали о проблемах в реализации совместных проектов с Кубой,0.251165
1,В Москве отметили снижение числа безработных,-0.728133
2,Дерипаска дважды за день раскритиковал политику ЦБ в отношении аграриев,-0.949828
3,Цены на сахар и масло ограничат до конца марта,-0.816628
4,"Структура банка ""Россия"" купила винодельческое предприятие ""Массандра""",-0.871995
5,Медведев оценил закон об удаленной работе,-0.296046
6,Глава МЭР рассказал о мерах стабилизации цен на продукты,-0.697207
7,Мишустин оценил идею ввести планку колебаний цен на социальные продукты,-0.558596
8,Россия вводит новые экспортные пошлины на зерно и пшеницу,-0.420460
9,Куда двинется рубль? Прогноз главного экономиста Альфа-банка,-0.785921


In [51]:
pd.set_option("display.max_colwidth", 160)

In [65]:
testdata = tfidf.transform(df["text"])

In [66]:
result2 = p.decision_function(testdata)

In [68]:
df["result"] = result2

In [56]:
tfidf.fit(df["text"])

f:\programming\python\economanalyzer\venv\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(min_df=3, ngram_range=(1, 3), smooth_idf=1,
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                sublinear_tf=1,
                tokenizer=<function ru_token at 0x0000018707181A68>, use_idf=1)